In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join
from glob import glob
from io import StringIO
from textwrap import dedent
import pickle

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

from src.single_sided_network_v2 import network_slh

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [3]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2018-06-12_mg_oct_single_excitation_three_pulses.json',
}
register_doit_as_IPython_magic()

In [4]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

In [5]:
! qdyn_version

QDYN 2.0dev revision dfd21683b898e790985b519daac35aeef7721f6c (master)
  features: no-check-cheby, no-check-newton, no-parallel-ham, use-mkl=sequential, use-mpi=intel, parallel-oct, backtraces, no-debug, no-no-ipo
  compiled with ifort on Mon Nov 13 14:54:23 2017 on host mlhpc2


$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# Compare variants of Krotov's method for optimization of a large-N Dicke state

In [6]:
ROOT = './data/oct_single_excitation_three_pulses/'

In [7]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    nt = row['nt']
    if frac_part == '00':
        if int(nt) == 500:
            T_str = "%s" % int_part
        else:
            T_str = "%s_nt%s" % (int_part, nt)
    else:
        if int(nt) == 500:
            T_str = '%s_%s' % (int_part, frac_part)
        else:
            T_str = '%s_%s_nt%s' % (int_part, frac_part, nt)
    rf = "%02dnodesT%s_%s_ntrajs%s" % (
        row['nodes'], T_str, row['variant'], row['n_trajs'])
    return join(ROOT, rf)

## SLH Models

For very large networks, the SLH models take a significant amount of time to calculate, so we'll want to cache them.

In [8]:
SLH_MODELS = {}

In [9]:
%mkdir -p {ROOT}

In [10]:
SLH_PICKLE_FILE = join(ROOT, "slh_models.pickle")

In [11]:
if os.path.isfile(SLH_PICKLE_FILE):
    with open(SLH_PICKLE_FILE, 'rb') as pickle_fh:
        SLH_MODELS.update(pickle.load(pickle_fh))

In [12]:
SLH_MODELS.keys()

dict_keys([])

## action wrappers

In [13]:
from src.doit_actions_v2 import (
    update_config, wait_for_clusterjob, run_traj_prop,
    write_rho_prop_custom_config, collect_rho_prop_errors)

In [14]:
def submit_optimization(rf, n_trajs, variant, task):
    """Asynchronously run optimization, using either the
    standard QDYN implementation (variant = 'normal') or the 'crossoct' QDYN
    implementation at a particular revision (variant='cross')

    Does not support use of MPI, so this should only run on a workstation with
    more cores than the total number of trajectories
    """
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    bodies = {
        'normal': dedent(r'''
            {module_load}
            cd $CLUSTERJOB_WORKDIR
            pwd
            OMP_NUM_THREADS={threads} \
              qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_ss .'''),
        'cross': dedent(r'''
            {module_load}
            module load prefix/crossoct

            qdyn_version | grep 66f6d24d2b07b9cc || {cancel_cmd}

            cd $CLUSTERJOB_WORKDIR
            pwd
            OMP_NUM_THREADS={threads} \
              qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_cross .''')}
    body = bodies[variant]
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    jobscript.n_trajs = str(int(n_trajs))
    jobscript.resources['ppn'] = 1 # not using MPI
    jobscript.resources['threads'] = min(int(n_trajs), 78)
    jobscript.threads = str(jobscript.resources['threads'])
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))


## custom uptodate routines

In [15]:
from src.qdyn_model_v2 import pulses_uptodate

## OCT task definitions

In [16]:
from src.qdyn_single_excitation_model_v1 import make_single_excitation_qdyn_oct_model
from src.dicke_half_model_v2 import dicke_guess_controls, num_vals
from sympy import Symbol

In [17]:
def write_dicke_single_model_three_pulses(
        slh, rf, T, theta=0, E0_cycles=2, mcwf=False, non_herm=False,
        lambda_a=1.0, J_T_conv=1e-4, iter_stop=5000, nt=None,
        max_ram_mb=100000, kappa=0.01, seed=None, observables='all',
        keep_pulses='prev', single_excitation_subspace=False,
        config='config'):
    """OCT model with identical pulses on all nodes"""
    n_nodes = slh.n_nodes
    # fix the controls to have one pulse for the initial node, on pulse for
    # the final node, and the same pulse for the intermediate nodes
    controls = dicke_guess_controls(
        slh=slh, theta=theta, T=T, E0_cycles=E0_cycles, nt=nt, kappa=kappa)
    control_1_sym = Symbol('Omega_1')
    control_1_pulse = controls[control_1_sym]
    control_N_sym = Symbol('Omega_%d' % n_nodes)
    control_N_pulse = controls[control_N_sym]
    control_i_sym = Symbol('Omega')
    control_i_pulse = controls[Symbol('Omega_2')]
    controls = {
        control_1_sym: control_1_pulse,
        control_i_sym: control_i_pulse,
        control_N_sym: control_N_pulse,}
    control_mapping = {}
    ctrl_syms = [Symbol('Omega_%d' % ind) for ind in range(1, n_nodes+1)]
    for i, ctrl_sym in enumerate(ctrl_syms):
        if i == 0:
            control_mapping[ctrl_sym] = control_1_sym
        elif i == len(ctrl_syms)-1:  # last
            control_mapping[ctrl_sym] = control_N_sym
        else:
            control_mapping[ctrl_sym] = control_i_sym
    slh = slh.substitute(control_mapping)
    # end
    assert single_excitation_subspace
    qdyn_model = make_single_excitation_qdyn_oct_model(
        slh, num_vals=num_vals(theta=theta, n_nodes=n_nodes, kappa=kappa),
        controls=controls, energy_unit='dimensionless',
        mcwf=mcwf, non_herm=non_herm, oct_target='dicke_1',
        lambda_a=lambda_a, iter_stop=iter_stop, keep_pulses=keep_pulses,
        J_T_conv=J_T_conv, max_ram_mb=max_ram_mb, seed=seed,
        observables=observables)
    qdyn_model.user_data['state_label'] = '10'  # for prop
    if (mcwf, non_herm) == (False, False):
        qdyn_model.user_data['rho'] = True
    qdyn_model.write_to_runfolder(rf, config=config)

In [18]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_nodes = row['nodes']
        nt = row['nt']
        try:
            slh = SLH_MODELS[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='open')
            SLH_MODELS[n_nodes] = slh
        if rf in jobs:
            continue
        mcwf = {  #   variant => whether to use MCWF
            'rho': False, 'nonherm': False, 'independent': True,
            'cross': True}
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                # write the density matrix propagation data
                (write_dicke_single_model_three_pulses, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=nt,
                    kappa=1.0, E0_cycles=2,
                    mcwf=False, non_herm=False, # this defines rho-prop
                    single_excitation_subspace=True,
                    config='config_rho')),
                # write the data for the optimization
                (write_dicke_single_model_three_pulses, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=nt,
                    kappa=1.0, E0_cycles=2, mcwf=mcwf[row['variant']],
                    non_herm=(row['variant'] != 'rho'),
                    lambda_a=row['lambda_a'], keep_pulses='all',
                    iter_stop=int(row['iter_stop']), J_T_conv=row['J_T_conv'],
                    single_excitation_subspace=True,
                    config='config'))],
            'targets': [join(rf, 'config_rho'), join(rf, 'config')],
            'uptodate': [True, ] # up to date if targets exist
        }
    for job in jobs.values():
        yield job

In [19]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row
    (adjusting lambda_a, iter_stop and J_T_conv only)"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop']),
                    J_T_conv=row['J_T_conv']))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [20]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    optimization_variant = {
        'rho': 'normal',
        'nonherm': 'normal',
        'independent': 'normal',
        'cross': 'cross',
    }
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ],
                 dict(n_trajs=row['n_trajs'],
                      variant=optimization_variant[row['variant']]))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [21]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

Additional tasks for evaluating the optimization success through a propagation
in Liouville space are defined farther below

##  OCT Submission

In [22]:
params_data_str = r'''
# nodes   T    nt  lambda_a      variant  n_trajs   iter_stop   J_T_conv
     20  50   500     0.001  independent        1        5000       1e-8
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=[
            'nodes', 'T', 'nt', 'lambda_a', 'variant', 'n_trajs', 'iter_stop',
            'J_T_conv'],
        converters={
            'nodes': int, 'T': float, 'nt': int, 'lambda_a': float,
            'variant': str, 'n_trajs': int, 'iter_stop': int,
            'J_T_conv': float})


In [23]:
print(params_df.to_string())

   nodes     T   nt  lambda_a      variant  n_trajs  iter_stop      J_T_conv
0     20  50.0  500     0.001  independent        1       5000  1.000000e-08


In [24]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(
    level=logging.DEBUG, filename='./data/oct_single_excitation_three_pulses.log')

In [ ]:
%doit -n 40 create_runfolder

.  create_runfolder:./data/oct_single_excitation_three_pulses/20nodesT50_independent_ntrajs1


In [ ]:
%doit -n 40 wait_for_optimization

-- create_runfolder:./data/oct_single_excitation_three_pulses/20nodesT50_independent_ntrajs1
.  update_runfolder:0
.  submit_optimization:0
.  wait_for_optimization:0


In [ ]:
with open(SLH_PICKLE_FILE, 'wb') as pickle_fh:
    pickle.dump(SLH_MODELS, pickle_fh)

## Evaluate error exactly from density matrix propagation

In [ ]:
RHO_PROP_ITERS = np.arange(6000, step=300)

In [ ]:
def task_rho_prop_create_custom_config():
    """For every optimized pulse, at intermediate iteration numbers,
    write a custom config file the will propagate this pulse"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for oct_pulse_file in glob(join(rf, 'pulse1.oct.dat.0*')):
            oct_iter = int(os.path.splitext(oct_pulse_file)[-1][1:])
            if oct_iter not in RHO_PROP_ITERS:
                continue  # skip; we don't want to propagate *all* pulses
            assert os.path.isfile(oct_pulse_file.replace('pulse1', 'pulse2'))
            config_out = 'config_rho.%08d' % oct_iter
            yield {
                'name': str(rf) + "/%s" % config_out,
                'actions': [
                    (write_rho_prop_custom_config,
                     [rf, oct_iter, config_out])],
                'targets': [join(rf, config_out)],
                'uptodate': [True, ] # up to date if target exists
        }

In [ ]:
def task_evaluate_rho_prop_error():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for custom_config in glob(join(rf, 'config_rho.0*')):
            oct_iter = int(os.path.splitext(custom_config)[-1][1:])
            yield {
                'name': str(rf) + "/%s" % custom_config,
                'actions': [
                    (run_traj_prop, [rf, ], dict(
                        n_trajs=1, n_procs=None, use_oct_pulses=False,
                        config=os.path.split(custom_config)[-1]))],
                'targets': [join(rf, 'P_target.dat.%08d' % oct_iter)],
                'file_dep': [custom_config],
                'uptodate': [True, ] # up to date if target exists
        }

In [ ]:
def task_collect_rho_prop_errors():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        P_target_obs_files = glob(join(rf, 'P_target.dat.0*'))
        target_file = join(rf, 'rho_prop_error.dat')
        yield {
            'name': target_file,
            'actions': [
                (collect_rho_prop_errors, P_target_obs_files,
                 dict(outfile=target_file))],
            'targets': [target_file],
            'file_dep': P_target_obs_files,
            'uptodate': [False, ]
            # We always create a new file
        }

In [ ]:
#%doit -n 78 rho_prop_create_custom_config

In [ ]:
#%doit -n 78 evaluate_rho_prop_error

In [ ]:
#%doit -n 78 collect_rho_prop_errors